# GOAL

Anonymize data from AR app to allow public sharing.
- AR Comments (OK)
- Countries (N/A)
- DSO (OK)
- ExchangeRates (N/A)
- Invoice Item Detail
- Invoices (OK)
- Items (OK)
- Link Table (N/A)
- Product Lines (OK)
- Subsidiaries (OK)

# PACKAGES

In [ ]:
import pandas as pd
from anonympy.pandas import dfAnonymizer
from anonympy.pandas.utils_pandas import available_methods
from anonympy.pandas.utils_pandas import fake_methods
import os
import gcsfs
import pickle
from random import shuffle

# PARAMETERS

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../secrets/gcp_qlik_key.json'
source_path='gs://qlik-demos-data/finance/in/'
destination_path='gs://qlik-demos-data/finance/out/'
pd_options = {"token": os.environ['GOOGLE_APPLICATION_CREDENTIALS']}
fs = gcsfs.GCSFileSystem(token=os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

# seeds and keys for anonymization
key = 'qlikrulesaboveallothers'
seed = 1001

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# files
    
dict_files = {
    'Countries': 'AR_Countries V1',
    "Invoice Item Detail":"AR_Invoice Item Detail V1",
    "Product Lines": "AR_Product Lines V1",
    "Invoices": "AR_Invoices V1",
    "Items":"AR_Items V1",
    "Comments":"AR_Comments V1",
    "DSO":"AR_DSO V1",
    "Link Table":"AR_Link Table V1",
    "Subsidiaries":"AR_Subsidiaries V1",
    "ExchangeRates":"AR_ExchangeRates V1"
    }

# FUNCTIONS

## noise_amount_column

In [ ]:
def noise_amount_column(original_column):
    noise_column=original_column.replace(".-","-0.",regex=True).astype('float')
    return noise_column.apply(lambda x: round(x*2/3+50000,1) if x>=0 else round(x*2/3-50000,1))

## scramble_column

In [ ]:
def scramble_column(original_column):
    scrambled_column=original_column.copy()
    def scramble_str(original_str):
        def return_number(number=0.3):
            return number
        
        scrambled_str=list(original_str)
        shuffle(scrambled_str,return_number)

        return "".join([str(item) for item in scrambled_str])
    return scrambled_column.apply(scramble_str)

## sequential_values_for_column

In [ ]:
def sequencial_values_for_column(original_column):
    columns_names={'index':'new',0:'original'}
    sequencial_values_for_column=pd.DataFrame(set(original_column)).reset_index().rename(columns=columns_names)
    sequencial_values_for_column=pd.merge(
        original_column,
        sequencial_values_for_column,
        left_on=original_column.name,
        right_on='original',
        how='left').drop(
            columns=['original',original_column.name])
    return sequencial_values_for_column.rename(columns={'new':original_column.name})

## fake_data_for_column

In [ ]:
def fake_data_for_column(original_column,fake_method):

    # create a 2 columns dataframe with the unique values from original_column twice
    df_exclusive_values=original_column.drop_duplicates()
    df_exclusive_values=pd.merge(df_exclusive_values,df_exclusive_values,how='inner',left_index=True,right_index=True,suffixes=('','_fake'))

    anon_exclusive_values=dfAnonymizer(df_exclusive_values)
    anon_exclusive_values.categorical_fake({original_column.name+'_fake':fake_method},seed=seed)
    
    fake_data_for_column=pd.merge(original_column,
        anon_exclusive_values.to_df(),
        how='left',
        on=original_column.name
        ).drop(columns=[original_column.name])
    return fake_data_for_column.rename(columns={original_column.name+'_fake':original_column.name})

# DATA ANONYMIZATION

## Subsidiaries

In [ ]:
# read original file from gcs
df_subsidiaries=pd.read_csv(source_path+dict_files['Subsidiaries']+'.csv',storage_options=pd_options)
df_subsidiaries['NetSuite Subsidiary ID']=df_subsidiaries['NetSuite Subsidiary ID'].astype('str')

print('original dataframe')
display(df_subsidiaries.head())

# anonymize dataframe
anon_subsidiaries = dfAnonymizer(df_subsidiaries)

# anon_subsidiaries.categorical_tokenization('%SubsidiaryCode',max_token_len=3,key=key)
anon_subsidiaries.categorical_fake({'Subsidiary':'company'},seed=seed)
anon_subsidiaries.column_suppression(['Is Attunity Subsidiary','VAT Registration Number'])
anon_subsidiaries.categorical_resampling(
    ['Subsidiary Currency Code','Subsidiary Region'],seed=seed)

print(anon_subsidiaries.info())

df_subsidiaries_anon=anon_subsidiaries.to_df()
df_subsidiaries_anon['NetSuite Subsidiary ID']=df_subsidiaries_anon['%SubsidiaryCode']
df_subsidiaries_anon['Subsidiary Legal Name']=df_subsidiaries_anon['Subsidiary']
df_subsidiaries_anon['Workday Subsidiary Name']=df_subsidiaries_anon['Subsidiary']
df_subsidiaries_anon['Subsidiary Region']=df_subsidiaries_anon['Subsidiary Region'].replace({'Technologies':'World'},inplace=False)

# merge original and anonymized dataframes
df_subsidiaries=df_subsidiaries.join(df_subsidiaries_anon,how='inner',lsuffix='_orig')
print('full dataframe')
display(df_subsidiaries.head())

# persist anonymized df to GCS
df_subsidiaries_anon.to_csv(destination_path+dict_files['Subsidiaries']+'.csv',index=False)

# persist mapping tables to GCS
# map_subsidiary_code = dict(zip(df_subsidiaries['%SubsidiaryCode_orig'], df_subsidiaries['%SubsidiaryCode']))
map_subsidiary_currency_code = dict(zip(df_subsidiaries['Subsidiary Currency Code_orig'], df_subsidiaries['Subsidiary Currency Code']))

# with fs.open(destination_path+'map_subsidiary_code.pickle', 'wb') as handle:
#     pickle.dump(map_subsidiary_code, handle, protocol=pickle.HIGHEST_PROTOCOL)
with fs.open(destination_path+'map_subsidiary_currency_code.pickle', 'wb') as handle:
    pickle.dump(map_subsidiary_currency_code, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_subsidiaries = anon_subsidiaries=df_subsidiaries_anon=map_subsidiary_code=map_subsidiary_currency_code=[]



## AR Comments

In [ ]:
# read original file from gcs
df_comments = pd.read_csv(source_path+dict_files['Comments']+'.csv',storage_options=pd_options)
#df_comments[['comment_date','comment_text']]=df_comments['%ARCommentKey'].str.split('|',expand=True,n=1)
#df_comments['comment_date']=pd.to_datetime(df_comments['comment_date']).dt.date
print('original dataframe')
display(df_comments.head())

# anonymize dataframe
anon_comments=dfAnonymizer(df_comments)
#anon_comments.column_suppression(['comment_text'])
anon_comments.categorical_tokenization(['%ARCommentKey'],max_token_len=10,key=key)
#anon_comments.datetime_noise('comment_date',seed=seed)

df_comments_anon=anon_comments.to_df()
df_comments_anon['AR Comments']=df_comments_anon['AR Comments'].apply(lambda x:0 if pd.isna(x) else 1)

anon_comments.info()

df_comments=df_comments.join(df_comments_anon,how='inner',lsuffix='_orig')
print('full dataframe')
display(df_comments.head())

#persist anonymized df to GCS
df_comments_anon.to_csv(destination_path+dict_files['Comments']+'.csv',index=False)

# persist mapping tables to GCS
map_comment_key = dict(zip(df_comments['%ARCommentKey_orig'], df_comments['%ARCommentKey']))
with fs.open(destination_path+'map_comment_key.pickle', 'wb') as handle:
    pickle.dump(map_comment_key, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_comments=df_comments_anon=map_comment_key=[]


## DSO

In [ ]:
# read original file from gcs
df_dso = pd.read_csv(source_path+dict_files['DSO']+'.csv')
#df_dso['NetSuite Extract DateTime']=pd.to_datetime(df_dso['NetSuite Extract DateTime'])

# split '%DSOKey' in period and subsidiary code to apply different anonymization
# df_dso[['period','subsidiary_code']]=df_dso['%DSOKey'].str.split('|',expand=True)
#df_dso['period']=pd.to_datetime(df_dso['period'])

print('original dataframe')
display(df_dso.head())

# read mapping tables from gcs

# with fs.open(destination_path+'map_subsidiary_code.pickle', 'rb') as handle:
#     map_subsidiary_code = pickle.load(handle)
with fs.open(destination_path+'map_subsidiary_currency_code.pickle', 'rb') as handle:
    map_subsidiary_currency_code = pickle.load(handle)

# anonymize dataframe
anon_dso=dfAnonymizer(df_dso)
#anon_dso.datetime_noise(['NetSuite Extract DateTime','period'],seed=seed)
anon_dso.info()

df_dso_anon=anon_dso.to_df()
df_dso_anon['Transaction Line Amount - Local']=noise_amount_column(
    df_dso_anon['Transaction Line Amount - Local'])
df_dso_anon['Transaction Line Amount - USD']=noise_amount_column(
    df_dso_anon['Transaction Line Amount - USD'])
# df_dso_anon['%DSOKey']=df_dso_anon[
#     'period'].dt.strftime("%Y-%m")+'|'+df_dso_anon['subsidiary_code'].map(map_subsidiary_code)
# df_dso_anon['From Currency Code']=df_dso_anon['From Currency Code'].map(map_subsidiary_currency_code)
# df_dso_anon['%DSOKey']=df_dso_anon[
#     'period']+'|'+df_dso_anon['subsidiary_code'].map(map_subsidiary_code)
df_dso_anon['From Currency Code']=df_dso_anon['From Currency Code'].map(map_subsidiary_currency_code)



# merge original and anonymized dataframes
df_dso=df_dso.join(df_dso_anon,how='inner',lsuffix='_orig')
print('full dataframe')
display(df_dso.head())

# persist anonymized df to GCS
df_dso_anon.to_csv(destination_path+dict_files['DSO']+'.csv',index=False)

# persist mapping tables to GCS
# map_dso_key = dict(zip(df_dso['%DSOKey_orig'], df_dso['%DSOKey']))
# with fs.open(destination_path+'map_dso_key.pickle', 'wb') as handle:
#     pickle.dump(map_dso_key, handle, protocol=pickle.HIGHEST_PROTOCOL)

# map_period=dict(zip(df_dso['period_orig'], df_dso['period']))
# with fs.open(destination_path+'map_period.pickle', 'wb') as handle:
#     pickle.dump(map_period, handle, protocol=pickle.HIGHEST_PROTOCOL)

# drop artificial columns created by splitting '%DSOKey'
# df_dso=df_dso.drop(columns=['period','subsidiary_code','period_orig','subsidiary_code_orig'])
# df_dso_anon=df_dso_anon.drop(columns=['period','subsidiary_code'])

df_dso=df_dso_anon=anon_dso=map_subsidiary_code=map_subsidiary_currency_code=map_dso_key=map_period=[]

## Invoices

In [ ]:
# read original file from gcs
df_invoice = pd.read_csv(source_path+dict_files['Invoices']+'.csv',low_memory=False)
# date_columns=['Date','Due Date','As Of Date','Rev. Rec. Start Date','Rev. Rec. End Date','Contract Item Start Date','Contract Item End Date']
#for column in date_columns:
#    df_invoice[column]=pd.to_datetime(df_invoice[column],errors='coerce')

string_columns=['Customer Code','PO Number','%ItemID']
for column in string_columns:
    df_invoice[column]=df_invoice[column].astype(str)

# create 2 columns to store the original values of the columns to be anonymized
# df_invoice[['document_id','item_id']]=df_invoice['%InvoiceItemKey'].str.split('|',expand=True,n=1)

print('original dataframe: ',df_invoice.shape)
display(df_invoice.head())

# read mapping tables from gcs
with fs.open(destination_path+'map_subsidiary_currency_code.pickle', 'rb') as handle:
    map_subsidiary_currency_code = pickle.load(handle)
# with fs.open(destination_path+'map_subsidiary_code.pickle', 'rb') as handle:
#     map_subsidiary_code = pickle.load(handle)
with fs.open(destination_path+'map_comment_key.pickle', 'rb') as handle:
    map_comment_key = pickle.load(handle)
# with fs.open(destination_path+'map_dso_key.pickle', 'rb') as handle:
#     map_dso_key = pickle.load(handle)


# anonymize dataframe
anon_invoice=dfAnonymizer(df_invoice)
anon_invoice.column_suppression(['Detail URL','Customer URL','%SummaryKey','Project Name','Credit Limit'])
# anon_invoice.datetime_noise(date_columns,seed=seed)

anon_invoice.categorical_tokenization(['PO Number'],max_token_len=10,key=key)
anon_invoice.categorical_resampling(['Country Code'],seed=seed)
anon_invoice.info()

df_invoice_anon=anon_invoice.to_df()
value_columns=[
    'Temp Transaction Amount',
    'Temp Amount Due (Foreign Currency)',
    'Open Balance',
    'Amount Due (Foreign Currency)',
    'Transaction Amount',
    'Remaining (m)',
    'Recognized Balance',
    'Remaining Deferred Balance',
    'Tax Value',
    'Recognized Balance (Foreign Currency)',
    'Remaining Deferred Balance (Foreign Currency)',
    'Tax Value (Foreign Currency)',
    'Recognized Balance (Local)',
    'Remaining Deferred Balance (Local)',
    'Tax Value (Local)'
    ]
for column in value_columns:
    df_invoice_anon[column]=noise_amount_column(df_invoice_anon[column])

# create fake data
df_invoice_anon['Customer Name']=fake_data_for_column(df_invoice_anon['Customer Name'],'company')
# df_invoice_anon['Sales Rep Name']=fake_data_for_column(df_invoice_anon['Sales Rep Name'],'name')
df_invoice_anon['Accounts Receivable Accountant']=fake_data_for_column(df_invoice_anon['Accounts Receivable Accountant'],'name')

# # anonymize %InvoiceItemKey and delete support fields
# df_invoice_anon['%InvoiceItemKey']=df_invoice_anon['document_id'].astype(str)+'|'+df_invoice_anon['item_id'].astype(str)
# df_invoice_anon=df_invoice_anon.drop(columns=['document_id','item_id'])
# df_invoice=df_invoice.drop(columns=['document_id','item_id'])

# replace values with mapped values
df_invoice_anon['Transaction Currency']=df_invoice_anon['Transaction Currency'].map(map_subsidiary_currency_code)
df_invoice_anon['%ARCommentKey']=df_invoice_anon['%ARCommentKey'].map(map_comment_key)
# df_invoice_anon['%DSOKey']=df_invoice_anon['%DSOKey'].map(map_dso_key)

# scramble values in 'Customer Code' and 'End User Code'
df_invoice_anon['Customer Code']=scramble_column(df_invoice_anon['Customer Code'].astype('str'))
df_invoice_anon['End User Code']=scramble_column(df_invoice_anon['End User Code'].astype('str'))
df_invoice_anon['Customer Original']=df_invoice_anon['Customer Code']+' '+df_invoice_anon['Customer Name']

# inherit values from other fields
df_invoice_anon['Customer']=df_invoice_anon['Customer Original']
df_invoice_anon['End User']=df_invoice_anon['End User Code']

df_invoice=df_invoice.join(df_invoice_anon,how='inner',lsuffix='_orig')
# display full dataframe
print('full dataframe',df_invoice.shape)
display(df_invoice.head())

#persist anonymized df to GCS
df_invoice_anon.to_csv(destination_path+dict_files['Invoices']+'.csv',index=False)
print('anon dataframe',df_invoice_anon.shape)

# persist mapping tables to GCS
# map_invoice_key=dict(zip(df_invoice['%InvoiceItemKey_orig'], df_invoice['%InvoiceItemKey']))
# with fs.open(destination_path+'map_invoice_key.pickle', 'wb') as handle:
#     pickle.dump(map_invoice_key, handle, protocol=pickle.HIGHEST_PROTOCOL)

#drop variables
map_subsidiary_currency_code=map_invoice_key=map_subsidiary_code=map_comment_key=map_dso_key=df_invoice=df_invoice_anon=anon_invoice=None

## Items

In [ ]:
# load original file from gcs
df_items=pd.read_csv(source_path+dict_files['Items']+'.csv')
string_columns=['%ItemID']
for column in string_columns:
    df_items[column]=df_items[column].astype(str)

display(df_items.head())

# anonymize dataframe
anon_items=dfAnonymizer(df_items)
anon_items.categorical_resampling(
    ['Item Type','Product Family'],seed=seed)

anon_items.info()

df_items_anon=anon_items.to_df()

# create fake data
categorical_fake_dict={'Item Name':'color_name','Bookings Group 1':'word','Bookings Group 2':'currency_name','Bookings Group 3':'job'}
for key,value in categorical_fake_dict.items():
    df_items_anon[key]=fake_data_for_column(df_items_anon[key],value)

# inherit values from anonymized fields
df_items_anon['Item Description']=df_items_anon['Item Name']
df_items_anon['Item']=df_items_anon['Item Name']
df_items_anon['Product Family']=df_items_anon['Product Family'].replace('Qonnect Fees','Misc',inplace=False)

# merge anonymized df with original df
df_items=df_items.join(df_items_anon,how='inner',lsuffix='_orig')
display(df_items.head())

# persist anonymized df to GCS
df_items_anon.to_csv(destination_path+dict_files['Items']+'.csv',index=False)

# drop variables
df_items=df_items_anon=anon_items=[]

## Product Lines

In [ ]:
# read data from GCS
df_product_lines=pd.read_csv(source_path+dict_files['Product Lines']+'.csv')
print('original dataframe')
display(df_product_lines.head())

# anonymize dataframe
anon_product_lines=dfAnonymizer(df_product_lines)
anon_product_lines.info()

df_product_lines_anon=anon_product_lines.to_df()

# create fake data
categorical_fake_dict={'Product Line 3':'state','Product Line 2':'free_email_domain'}
for key,value in categorical_fake_dict.items():
    df_product_lines_anon[key]=fake_data_for_column(df_product_lines_anon[key],value)

map_product_line1={
    'Analytics':'Go to Market','Data Integration':'Operations','Non-Product Invoicing':'Others','Professional Services' : 'Support functions'}
df_product_lines_anon['Product Line 1']=df_product_lines_anon['Product Line 1'].map(map_product_line1)

# merge anonymized df with original df
df_product_lines=df_product_lines.join(df_product_lines_anon,how='inner',lsuffix='_orig')
display(df_product_lines.head())

# persist anonymized df to GCS
df_product_lines_anon.to_csv(destination_path+dict_files['Product Lines']+'.csv',index=False)

# persist mapping tables to GCS
map_product_line3=dict(zip(df_product_lines['Product Line 3_orig'], df_product_lines['Product Line 3']))
with fs.open(destination_path+'map_product_line3.pickle', 'wb') as handle:
    pickle.dump(map_product_line3, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_product_lines=df_product_lines_anon=anon_product_lines=map_product_line3=None

## Invoice Item Detail

In [ ]:
# read data from GCS
df_invoice_item=pd.read_csv(source_path+dict_files['Invoice Item Detail']+'.csv')
print('original dataframe')
display(df_invoice_item.head())

# read mapping tables from GCS
# with fs.open(destination_path+'map_invoice_key.pickle', 'rb') as handle:
#     map_invoice_key = pickle.load(handle)
with fs.open(destination_path+'map_product_line3.pickle', 'rb') as handle:
    map_product_line3 = pickle.load(handle)

#anonymize dataframe
anon_invoice_item=dfAnonymizer(df_invoice_item)
anon_invoice_item.column_suppression(['Created By'])
anon_invoice_item.info()

df_invoice_item_anon=anon_invoice_item.to_df()

# df_invoice_item_anon['%InvoiceItemKey']=df_invoice_item_anon['%InvoiceItemKey'].map(map_invoice_key)
df_invoice_item_anon['Product Line 3']=df_invoice_item_anon['Product Line 3'].map(map_product_line3)

# merge anonymized df with original df
df_invoice_item=df_invoice_item.join(df_invoice_item_anon,how='inner',lsuffix='_orig')
display(df_invoice_item.head())

# persist anonymized df to GCS
df_invoice_item_anon.to_csv(destination_path+dict_files['Invoice Item Detail']+'.csv',index=False)

df_invoice_item_anon=df_invoice_item=anon_invoice_item=map_invoice_key=map_product_line3=None